In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import pickle

from paws.data_to_mp4 import encode_tensor_to_video, decode_video_to_tensor

from paws.model_to_grid import make_source_2d, class_grid_to_medium_grid, make_sensor_2d, simulation_2d, point_cloud_voxelization

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dir_path = "C:\\replica\\apartment_1"


ply_path = os.path.join(dir_path,"habitat","mesh_semantic.ply")
print(ply_path)
json_path = os.path.join(dir_path,"habitat","info_semantic.json")
print(json_path)

C:\replica\apartment_1\habitat\mesh_semantic.ply
C:\replica\apartment_1\habitat\info_semantic.json


In [4]:

Nx = 256
Ny = 256

dx = 2e-2
dy = 2e-2



cls_grid = np.zeros([Nx,Ny],dtype=int)


medium = class_grid_to_medium_grid(cls_grid,{})

source = make_source_2d(128,128,2,Nx,Ny,5)

sensor = make_sensor_2d(np.ones([Nx,Ny],dtype=bool))



sensor_data = simulation_2d(Nx,Ny,dx,dy,source,medium,sensor)




# save path of project
base_dir = os.getcwd()
save_dir = os.path.join(base_dir,"data")

save_base = os.path.basename(dir_path)


pressure_dist = sensor_data["p"].copy()
pressure_dist = pressure_dist.reshape((pressure_dist.shape[0], Nx, Ny))

save_filename = "test_output"

save_path = os.path.join(save_dir,save_filename)

#save pressure distribution
video_path = save_path + "_pressure.mp4"
meta_path = save_path + "_pressure.pkl"




v_scales = encode_tensor_to_video(
    tensor=pressure_dist, 
    video_path=video_path,
)
# v_mins: (T,)
# v_maxs: (T,)
###


print(v_scales)

meta = {
    "v_scales": v_scales
}
pickle.dump(meta, open(meta_path, "wb"))
print("Write out meta to {}".format(meta_path))

#save medium data
medium_save_pth = save_path + "_medium.npy"
medium_data = {"sound_speed":medium.sound_speed,
                "density":medium.density}

np.save(medium_save_pth,medium_data)

+---------------------------------------------------------------+
|                  kspaceFirstOrder-CUDA v1.3                   |
+---------------------------------------------------------------+
| Reading simulation configuration:                        Done |
| Selected GPU device id:                                     0 |
| GPU device name:                NVIDIA GeForce RTX 4070 SUPER |
| Number of CPU threads:                                      1 |
| Processor name:         13th Gen Intel(R) Core(TM) i5-13600KF |
+---------------------------------------------------------------+
|                      Simulation details                       |
+---------------------------------------------------------------+
| Domain dimensions:                                  256 x 256 |
| Medium type:                                               2D |
| Simulation time steps:                                   1207 |
+---------------------------------------------------------------+
|         